## Draw 3D Points and Lines

We usually draw points and lines in the 3D world to debug when developing new features and fixing bugs. There two types of visualization tools can be used: **line** and **point**. In this section, we will show how to use these tools. First of all, let's make an environment.

In [1]:
from metadrive.envs.base_env import BaseEnv
import os
render = not os.getenv('TEST_DOC')

env = BaseEnv(dict(use_render=render))

[INFO] MetaDrive version: 0.4.1.2
[INFO] Sensors: [lidar: Lidar(50,), side_detector: SideDetector(), lane_line_detector: LaneLineDetector(), dashboard: DashBoard(), main_camera: MainCamera(1200, 900)]
[INFO] Render Mode: onscreen
[WARNING] You are using DummyObservation which doesn't collect information from the environment. (observation_base.py:42)


### Line
A line can be represented by a list of points. 

In [2]:
line=[[x,x,0.5] for x in range(10)] # generate a line for test

